In [0]:
class batch_count:
    def __init__(self):
        self.base_dir="/FileStore/project1"
    
    def read_data(self):
        return spark.read.format("text").option("linSep",".").load(f"{self.base_dir}/data/text")
    # spark.read.format("text").option("linSep",".").load(f"{self.base_dir}/data/text")
    
    def clean_data(self,raw_df):
        from pyspark.sql.functions import trim,lower,split,explode,col
        lines=raw_df.select(explode(split(raw_df.value," ")).alias("words"))
        return (lines.select(lower(trim("words")).alias("words")).where(col("words").isNotNull() & col("words").rlike("[a-z]")))
    
    def countRes(self,clean_df):
        from pyspark.sql.functions import count
        return (clean_df.groupBy("words").count())

    def writeResToDelta(self,df):
        (df.write\
                .format("delta")\
                .mode("overwrite")\
                .saveAsTable("word_count_table"))
        
    def computeBatch(self):
        print(f"\tExecuting word count...",end='')
        raw_df=self.read_data()
        clean_df=self.clean_data(raw_df)
        df=self.countRes(clean_df)
        self.writeResToDelta(df)
        print("Done!\n")










In [0]:
class stream_count:
    def __init__(self):
        self.base_dir="/FileStore/project1"
    
    def read_data(self):
        return spark.readStream.format("text").option("linSep",".").load(f"{self.base_dir}/data/text")
    # spark.read.format("text").option("linSep",".").load(f"{self.base_dir}/data/text")
    
    def clean_data(self,raw_df):
        from pyspark.sql.functions import trim,lower,split,explode,col
        lines=raw_df.select(explode(split(raw_df.value," ")).alias("words"))
        return (lines.select(lower(trim("words")).alias("words")).where(col("words").isNotNull() & col("words").rlike("[a-z]")))
    
    def countRes(self,clean_df):
        from pyspark.sql.functions import count
        return (clean_df.groupBy("words").count())

    def writeResToDelta(self,df):
        return (df.writeStream\
                .format("delta")\
                .option("checkpointLocation",f"{self.base_dir}/checkpoint/word_count")\
                .outputMode("complete")\
                .toTable("word_count_table"))
        
    def computeStream(self):
        print(f"\tExecuting word count...",end='')
        raw_df=self.read_data()
        clean_df=self.clean_data(raw_df)
        df=self.countRes(clean_df)
        sQuery=self.writeResToDelta(df)
        print("Done!\n")
        return sQuery










In [0]:
dbutils.fs.mkdirs("/FileStore/project1/datasets")

Out[10]: True